In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageOps
import numpy as np
import sklearn.model_selection
import os
from google.colab import drive

In [ ]:
#For the program to work, data needs to be downloaded (https://drive.google.com/file/d/1hgXutPsaBE7yhqKjMwKy1OtvVQHV4XXk/view?usp=sharing - this is updated file with new photos).
#It was found that some problems with uploading big amounts of photos my cause the colab to stop working. The best way to upload photos, is to download the .zip file and upload it
#to google drive and use colab api to get the data from the .zip file.
#Please, upload the zip file to main directory of your google drive (if not possible, then please change code in next code block).

def createDirectories():
  #Directories structure (it is checked if directory already exists in order not to erase uploaded images):
  drive.mount('drive')
  
  if not os.path.isdir("processedData"):
    os.mkdir("processedData")
    
createDirectories()

In [ ]:
#This command will extract all the files from .zip file and move them to new directories.
#Please check if the location of the file in this code is correct with the location where you uploaded the code on personal google drive.
!unzip drive/MyDrive/updatedData.zip

In [ ]:
def preprocessImages(img_cols, img_rows):
  #Load data, modify files and save them (COVID)
  for i, fileName in enumerate(os.listdir("updatedData/CT_COVID")):
    image = Image.open("updatedData/CT_COVID/{}".format(fileName)) #load
    resizedImage = image.resize((img_cols, img_rows)) #resize
    grayImage = ImageOps.grayscale(resizedImage) #grayscale
    grayImage.save("processedData/COVID_{}.jpg".format(i)) #save

  #Load data, modify files and save them (NonCOVID)
  for i, fileName in enumerate(os.listdir("updatedData/CT_NonCOVID")):
    image = Image.open("updatedData/CT_NonCOVID/{}".format(fileName)) #load
    resizedImage = image.resize((img_cols, img_rows)) #resize
    grayImage = ImageOps.grayscale(resizedImage) #grayscale
    grayImage.save("processedData/NonCOVID_{}.jpg".format(i)) #save

In [ ]:
def createNumpyData(num_classes):
  xData = []
  yData = []
  #Load processed data and append values to arrays
  for fileName in os.listdir("processedData"):
    image = Image.open('processedData/{}'.format(fileName))
    xData.append(np.asarray(image))
    if fileName[0] == "C":
      yData.append(1)
    else:
      yData.append(0)
  
  #Convert python array to numpy array
  xNumpyData = np.array(xData)
  yNumpyData = np.array(yData)
  
  #Convert pixel values to values between 0 and 1
  xNumpyData = xNumpyData.astype('float32')
  xNumpyData /= 255
  
  #Assign classes for yData
  yNumpyData = np_utils.to_categorical(yNumpyData, num_classes)

  return xNumpyData, yNumpyData

In [ ]:
def splitData(xNumpyData, yNumpyData, test_size, img_cols, img_rows):
  #Split the data
  x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(xNumpyData, yNumpyData, test_size=0.1, random_state=42)
  #Reshape the xData (gray scale is used so only 1 number needed to describe each pixel)
  x_train = x_train.reshape(x_train.shape[0], img_cols, img_rows, 1)
  x_test = x_test.reshape(x_test.shape[0], img_cols, img_rows, 1)
  input_shape = (img_cols, img_rows, 1)

  return x_train, x_test, y_train, y_test, input_shape

In [ ]:
#Specify image size and process the images to vectors
img_cols, img_rows = 140, 100
num_classes = 2

preprocessImages(img_cols, img_rows)
xNumpyData, yNumpyData = createNumpyData(num_classes)

In [ ]:
def createDeepModel(input_shape, num_classes):
  #Create deep model
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape, padding='same')) 
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(MaxPooling2D(pool_size=(2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(MaxPooling2D(pool_size=(2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(MaxPooling2D(pool_size=(2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(num_classes, activation='softmax')) 
  model.summary()
  model.compile(loss=binary_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
  return model

In [ ]:
def trainModelAugumented(model, x_train, y_train, x_test, y_test, batch_size, epochs):    
  datagen = ImageDataGenerator(
      #Rotate images up to specified value
      rotation_range=15,
      #Random vertical and horizontal shift up to specified value
      width_shift_range=0.1,
      height_shift_range=0.1,
      #Zoom up to specified value
      zoom_range=0.2,
      #Allow horizontal flip but not vertical
      horizontal_flip=True,
      vertical_flip=False)

  datagen.fit(x_train)
  #Apply augumentation and train the model
  model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test))
  score = model.evaluate(x_test, y_test, verbose=0)
  return score

In [ ]:
def createModel(input_shape, num_classes): 
  #Create model
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2))) 
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(num_classes, activation='softmax'))
  model.summary()
  model.compile(loss=binary_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
  return model

In [ ]:
def trainModel(model, x_train, y_train, x_test, y_test, batch_size, epochs):
  model.fit(x_train, y_train, batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test))
  score = model.evaluate(x_test, y_test, verbose=0)
  return score

In [ ]:
#Specify test size and split the data
test_size = 0.1
x_train, x_val, y_train, y_val, input_shape = splitData(xNumpyData, yNumpyData, test_size, img_cols, img_rows)

In [ ]:
#Specify learning parameters and fit the model
batch_size = 32
epochs = 12

model = createModel(input_shape, num_classes)
trainModel(model, x_train, y_train, x_val, y_val, batch_size, epochs)

In [ ]:
#Save model for GUI implementation
model.save("model")

In [ ]:
#ZIP the model so it can be downloaded
!zip -r model.zip model/